In [2]:
import os
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, random_split
import pytorch_lightning as pl

from nuscenes.eval.prediction.splits import get_prediction_challenge_split

from nuscenes.prediction.models.backbone import ResNetBackbone
from nuscenes.prediction.models.mtp import MTP
from nuscenes.prediction.models.covernet import CoverNet

import matplotlib.pyplot as plt
%matplotlib inline

from nuscenes.prediction.input_representation.static_layers import StaticLayerRasterizer
from nuscenes.prediction.input_representation.agents import AgentBoxesWithFadedHistory
from nuscenes.prediction.input_representation.interface import InputRepresentation
from nuscenes.prediction.input_representation.combinators import Rasterizer

In [2]:
class PredictionModel(pl.LightningModule):

    def __init__(self):
        super().__init__()
        backbone = ResNetBackbone('resnet50')
        
        # Note that the value of num_modes depends on the size of the lattice used for CoverNet.
        self.covernet = CoverNet(backbone, num_modes=64)
        # mtp = MTP(backbone, num_modes=2)
        
        # How to pass the nuscenes data helper to the model module?
        self.helper = PredictHelper(nuscenes)
        
        # Image representation
        static_layer_rasterizer = StaticLayerRasterizer(helper)
        agent_rasterizer = AgentBoxesWithFadedHistory(helper, seconds_of_history=1)
        self.mtp_input_representation = InputRepresentation(static_layer_rasterizer, agent_rasterizer, Rasterizer())

    def forward(self, x):
        # in lightning, forward defines the prediction/inference actions
        logits = self.covernet(x)
        return logits

    def training_step(self, batch, batch_idx):
        # training_step defined the train loop.
        # It is independent of forward
        x, y = batch
        x = x.view(x.size(0), -1)

        # Needs to be set by the batch
        instance_token_2, sample_token_2 = mini_train[5].split("_")
        instance_token_img, sample_token_img = 'bc38961ca0ac4b14ab90e547ba79fbb6', '7626dde27d604ac28a0240bdd54eba7a'
        img = self.mtp_input_representation.make_input_representation(instance_token_img, sample_token_img)
        
        agent_state_vector = torch.Tensor([[helper.get_velocity_for_agent(instance_token_img, sample_token_img),
                                    helper.get_acceleration_for_agent(instance_token_img, sample_token_img),
                                    helper.get_heading_change_rate_for_agent(instance_token_img, sample_token_img)]])

        image_tensor = torch.Tensor(img).permute(2, 0, 1).unsqueeze(0)
        
        logits = self.covernet(image_tensor, agent_state_vector)
        loss = F.mse_loss(x_hat, x)
        # Logging to TensorBoard by default
        self.log('train_loss', loss)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

In [3]:
DATAROOT = '/home/mitkina/git-repos/Waymo/data/sets/nuscenes'

# nuscenes = NuScenes('v1.0-mini', dataroot=DATAROOT)
nuscenes = NuScenes('v1.0-trainval', dataroot=DATAROOT)

0it [00:00, ?it/s]

Extracting /home/mitkina/git-repos/Waymo/nuscenes-devkit/python-sdk/tutorials/MNIST/raw/train-images-idx3-ubyte.gz to /home/mitkina/git-repos/Waymo/nuscenes-devkit/python-sdk/tutorials/MNIST/raw


0it [00:00, ?it/s]

Extracting /home/mitkina/git-repos/Waymo/nuscenes-devkit/python-sdk/tutorials/MNIST/raw/train-labels-idx1-ubyte.gz to /home/mitkina/git-repos/Waymo/nuscenes-devkit/python-sdk/tutorials/MNIST/raw


0it [00:00, ?it/s]

Extracting /home/mitkina/git-repos/Waymo/nuscenes-devkit/python-sdk/tutorials/MNIST/raw/t10k-images-idx3-ubyte.gz to /home/mitkina/git-repos/Waymo/nuscenes-devkit/python-sdk/tutorials/MNIST/raw


0it [00:00, ?it/s]

Extracting /home/mitkina/git-repos/Waymo/nuscenes-devkit/python-sdk/tutorials/MNIST/raw/t10k-labels-idx1-ubyte.gz to /home/mitkina/git-repos/Waymo/nuscenes-devkit/python-sdk/tutorials/MNIST/raw
Processing...
Done!


/home/mitkina/anaconda3/envs/bp/lib/python3.6/site-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
# dataset = MNIST(os.getcwd(), download=True, transform=transforms.ToTensor())

train_dataset = get_prediction_challenge_split("train", dataroot=DATAROOT)
val_dataset = get_prediction_challenge_split("validation", dataroot=DATAROOT)

train_loader = DataLoader(dataset)

    dataloader = torch.utils.data.DataLoader(
        dataset,
        batch_size=input_cfg.batch_size * num_gpu,
        shuffle=True,
        num_workers=input_cfg.preprocess.num_workers * num_gpu,
        pin_memory=False,
        collate_fn=collate_fn,
        worker_init_fn=_worker_init_fn,
        drop_last=not multi_gpu)
    eval_dataloader = torch.utils.data.DataLoader(
        eval_dataset,
        batch_size=eval_input_cfg.batch_size, # only support multi-gpu train
        shuffle=False,
        num_workers=eval_input_cfg.preprocess.num_workers,
        pin_memory=False,
        collate_fn=merge_second_batch)

In [ ]:
# init model
autoencoder = LitAutoEncoder()

# most basic trainer, uses good defaults (auto-tensorboard, checkpoints, logs, and more)
# trainer = pl.Trainer(gpus=8) (if you have GPUs)
trainer = pl.Trainer(gpus=1)
trainer.fit(autoencoder, train_loader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type       | Params
---------------------------------------
0 | encoder | Sequential | 50.4 K
1 | decoder | Sequential | 51.2 K
---------------------------------------
101 K     Trainable params
0         Non-trainable params
101 K     Total params
0.407     Total estimated model params size (MB)
/home/mitkina/anaconda3/envs/bp/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Training: 0it [00:00, ?it/s]

In [ ]:
# test
trainer.test(datamodule=dm)

In [ ]:
model = LitModel.load_from_checkpoint(path)

# # load the ckpt
# ckpt = torch.load('path/to/checkpoint.ckpt')

# # equivalent to the above
# model = LitModel()
# model.load_state_dict(ckpt['state_dict'])

In [ ]:
class NuScenesDataModule(LightningDataModule):

      def __init__(self, batch_size=32, dataroot):
          super().__init__()
          self.batch_size = batch_size
          self.dataroot = dataroot
            
          self.nuscenes = NuScenes('v1.0-trainval', dataroot=self.dataroot)
          self.helper = PredictHelper(self.nuscenes)

      # When doing distributed training, Datamodules have two optional arguments for
      # granular control over download/prepare/splitting data:

      # OPTIONAL, called only on 1 GPU/machine
      # write to disk
#       def prepare_data(self):
#           MNIST(os.getcwd(), train=True, download=True)
#           MNIST(os.getcwd(), train=False, download=True)

      # OPTIONAL, called for every GPU/machine (assigning state is OK)
      def setup(self, stage: Optional[str] = None):
          # transforms
          transform=transforms.Compose([
              transforms.ToTensor(),
              transforms.Normalize((0.1307,), (0.3081,))
          ])
          # split dataset
          if stage == 'fit':
              train = get_prediction_challenge_split("train", dataroot=self.dataroot)
              self.train, self.val = random_split(train, [int(0.9*len(train), len(train) - int(0.9*len(train)])
#               mnist_train = MNIST(os.getcwd(), train=True, transform=transform)
#               self.mnist_train, self.mnist_val = random_split(mnist_train, [55000, 5000])
# #           if stage == 'validate':
# #               mnist_train = MNIST(os.getcwd(), train=True, transform=transform)
#               self.mnist_train, self.mnist_val = random_split(mnist_train, [55000, 5000])
          if stage == 'test':
#               self.mnist_test = MNIST(os.getcwd(), train=False, transform=transform)
                self.test = get_prediction_challenge_split("val", dataroot=self.dataroot)
#           if stage == 'predict':
#               self.mnist_test = MNIST(os.getcwd(), train=False, transform=transform)

      # return the dataloader for each split
      def train_dataloader(self):
          mnist_train = DataLoader(self.train, batch_size=self.batch_size)
          return mnist_train

      def val_dataloader(self):
          mnist_val = DataLoader(self.val, batch_size=self.batch_size)
          return mnist_val

      def test_dataloader(self):
          mnist_test = DataLoader(self.test, batch_size=self.batch_size)
          return mnist_test